In [ ]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import helper

import matplotlib as mpl
import pickle
from copy import deepcopy
import time
#from graphviz import Digraph
import itertools
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
import seaborn as sns
from scipy.stats import multivariate_normal

N = 2**2 # particleok szama
sigma_r = .5
sig_r_model = .5

Tx = 5
Ty = 5
EM_SIZE = 3

In [ ]:
def sokatmondo_adat(Tx=Tx,Ty=Ty):
  alpha = 90 #x-esek eloszor
  sigma_r = .3
  gamma = helper.gamma_from_alpha(alpha)
  #Tx = 3
  z_prior = tfd.MultivariateNormalDiag(loc=[-1,1], scale_diag=[.3,.3]);
  z = np.array(z_prior.sample(Tx))
  r_noise = tfd.Normal(0, .001).sample(Tx)
  r_mean = tf.reduce_sum(tf.multiply(gamma,z),1)
  r = r_mean + r_noise
  datax = {'z':z,'r':r}

  alpha = 0 #utana y-osok
  gamma = helper.gamma_from_alpha(alpha)
  #Ty = 3
  z_prior = tfd.MultivariateNormalDiag(loc=[-1,1], scale_diag=[.3,.3]);
  z = np.array(z_prior.sample(Ty))
  r_noise = tfd.Normal(0, .001).sample(Ty)
  r_mean = tf.reduce_sum(tf.multiply(gamma,z),1)
  r = r_mean + r_noise
  datay = {'z':z,'r':r}

  data = concatenate_data(datax, datay)
  z = data['z']
  r = np.array(data['r'])

  xylabels = ['x']*Tx + ['y']*Ty
  return z, r, xylabels

def concatenate_data(data1, data2):
  z = np.concatenate((data1['z'], data2['z']), 0)
  r = np.concatenate((np.array(data1['r']), np.array(data2['r'])))
  return {'z': z, 'r': r}

def plot_data_xy_labels(data, labels):
    plt.scatter(*data['z'].T,c=data['r'])
    plt.gca().set_aspect('equal')
    plt.colorbar()
    plt.xlabel('z_1')
    plt.ylabel('z_2')
    plt.axhline(y = 0)
    plt.axvline(x = 0)
    for label, x, y in zip(labels, data['z'][:, 0], data['z'][:, 1]):
        plt.annotate(
            label,
            xy=(x, y), xytext=(-20, 20),
            textcoords='offset points', ha='right', va='bottom',
            bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
            arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

In [ ]:
z, r, xylabels = sokatmondo_adat(5,5)
true_data = {'z':z, 'r':r}
#plot_data_xy_labels(true_data, xylabels)

In [ ]:
plot_data_xy_labels(true_data, xylabels)

In [ ]:
data_x = {'z':true_data['z'][:Tx], 'r':true_data['r'][:Tx]}

In [ ]:
plot_data_xy_labels(data_x, xylabels)

In [ ]:
def generate_data_from_posterior_samples(N=100, gamma=0, z_prior_type='uniform', sigma_z_prior=1, r_bias=0, sigma_reward=0.1, sigma_bias=0):
    if z_prior_type == 'normal':
        z_prior = tfd.MultivariateNormalDiag(loc=[0,0], scale_diag=[sigma_z_prior,sigma_z_prior]);
    elif z_prior_type == 'uniform':
        z_prior = tfd.Uniform([-sigma_z_prior,-sigma_z_prior],[sigma_z_prior,sigma_z_prior])
    elif z_prior_type == 'informative':
        z_prior = tfd.MultivariateNormalDiag(loc=[-1,1], scale_diag=[.3,.3]);


    z = np.array(z_prior.sample(N))

    r_noise = tfd.Normal(0, sigma_reward).sample(N)
    r_mean = tf.cast(tf.reduce_sum(tf.multiply(gamma,z),1),dtype=tf.float32) + r_bias
    r = r_mean + r_noise

    return z[0],r[0]
  
def generate_dream_data_from_posterior_samples(samples,z_prior_type):
  z_samples = []
  r_samples = []

  for gamma_sample in samples:
      # generate a single datapoint for each gamma sample
      z,r = generate_data_from_posterior_samples(N=1, gamma=gamma_sample, z_prior_type=z_prior_type, sigma_reward=sig_r_model)
      z_samples.append(z)
      r_samples.append(r)

  z_samples = np.array(z_samples)
  r_samples = np.array(r_samples)

  return {'z': np.array(z_samples), 'r': np.array(r_samples)}

def generate_dream_data_set(posterior, T=10, N=2, z_prior_type='uniform'):
  '''generates multiple dream data sets'''
  dream_data_sets = []
  for i in range(N):
    samples = posterior.sample(T)
    dream_data = generate_dream_data_from_posterior_samples(samples, z_prior_type)
    dream_data_sets.append(dream_data)
  return dream_data_sets

### Compute mllh for 1 task and 2 task models on dream data

In [ ]:
# mllh on 2D 2 task model
# these are from particle_filter.ipynb but is it correct to just mllh?

from itertools import chain, combinations
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def model_marginal_llh_analytic(zs, rs, sigma_r, Sigma_0 = np.array([[1., 0.], [0., 1.]]), model = '2d'):
    # this is the model marginal likelihood function
    # it is validated through 'trial_nonorm_posterior_set_transformed'
    # from that function the only step fowrad is to leave the normal in gamma (the gamma posterior) since gamma is marginalized out
    if zs.size != 0:
      T = np.size(zs,0)
      if model == '2d':
        assert not np.isscalar(Sigma_0), 'Sigma_0 must be a 2-dimensional array'
        detSigma_0 = np.linalg.det(Sigma_0)
        Sigma_i_star_invs = []
        Sigma_i_invs = []
        mu_is = []
        y = 1/(2*np.pi)/np.sqrt(np.linalg.det(Sigma_0))
        for t in range(T):
            z = zs[t]
            r = rs[t]
            Sigma_i_star_inv = np.array([[z[0]**2/sigma_r**2, z[0]*z[1]/sigma_r**2],[z[0]*z[1]/sigma_r**2, z[1]**2/sigma_r**2]])
            Sigma_i_star_invs.append(Sigma_i_star_inv)
            if t==0:
                Sigma_i_inv = Sigma_i_star_inv + np.linalg.inv(Sigma_0)
            else:
                Sigma_i_inv = Sigma_i_star_inv + Sigma_i_invs[t-1]
            Sigma_i_invs.append(Sigma_i_inv)
            Sigma_i = np.linalg.inv(Sigma_i_inv)
            if t==0:
                mu_i = Sigma_i.dot(z*r/sigma_r**2)
            else:
                mu_i = Sigma_i.dot(z*r/sigma_r**2 + Sigma_i_invs[t-1].dot(mu_is[t-1]))
            mu_is.append(mu_i)
            y = y * multivariate_normal.pdf(r, mean = 0, cov = sigma_r**2)
        y = y / multivariate_normal.pdf(mu_i, mean = np.array([0,0]), cov = Sigma_i)
      else:
        '''
        Sigma_0 is the standard deviation of the gamma prior
        '''
        assert np.isscalar(Sigma_0), 'Sigma_0 must be scalar'
        if model == 'x':
          integral_dim = 1
        else:
          integral_dim = 0

        Sigma_i_star_invs = []
        Sigma_i_invs = []
        mu_is = []
        y = 1/(np.sqrt(2*np.pi))/Sigma_0
        for t in range(T):
            z = zs[t]
            r = rs[t]
          
            Sigma_i_star_inv = z[integral_dim]**2/sigma_r**2
            Sigma_i_star_invs.append(Sigma_i_star_inv)
            if t==0:
                Sigma_i_inv = Sigma_i_star_inv + 1/Sigma_0**2
            else:
                Sigma_i_inv = Sigma_i_star_inv + Sigma_i_invs[t-1]
            Sigma_i_invs.append(Sigma_i_inv)
            Sigma_i = 1/Sigma_i_inv
            if t==0:
                mu_i = Sigma_i * z[integral_dim]*r/sigma_r**2
            else:
                mu_i = Sigma_i * (z[integral_dim]*r/sigma_r**2 + Sigma_i_invs[t-1]*mu_is[t-1])
            mu_is.append(mu_i)
            y = y * multivariate_normal.pdf(r, mean = 0, cov = sigma_r**2)
        y = y / multivariate_normal.pdf(mu_i, mean = 0.0, cov = Sigma_i)

      return y
    else:
      return 1.

def model_marginal_llh_analytic_2x2D(z, r, sigma_r, Sigma_0_2D = np.array([[1., 0.], [0., 1.]]), verbose = True):
  T = z.shape[0]
  
  indices = np.arange(T)
  index_subsets = list(powerset(indices))

  mmllh_accumulator = 0.
  if verbose:
    pbar = tf.keras.utils.Progbar(len(index_subsets))
  for index_subset in index_subsets:
    z1 = z[list(index_subset)]
    r1 = r[list(index_subset)]
    
    complementer_subset = [item for item in indices if item not in index_subset]
    
    z2 = z[complementer_subset]
    r2 = r[complementer_subset]
    
    mmllh_accumulator += model_marginal_llh_analytic(z1, r1, sigma_r, Sigma_0 = Sigma_0_2D, model = '2d') \
    * model_marginal_llh_analytic(z2, r2, sigma_r, Sigma_0 = Sigma_0_2D, model = '2d')
    
    if verbose:
      pbar.add(1)
      
  mmllh_accumulator /= 2**T
  return mmllh_accumulator

In [ ]:
data_x = {'z':true_data['z'][:Tx], 'r':true_data['r'][:Tx]}
data_EM = {'z':true_data['z'][Tx:Tx+EM_SIZE], 'r':true_data['r'][Tx:Tx+EM_SIZE]}
data_fit = {'z':np.concatenate([data_x['z'], data_EM['z']]), 'r':np.concatenate([data_x['r'], data_EM['r']])}

In [ ]:
# Fit 2D 1 task model
posterior_params = helper.gamma_posterior_analytic(data_fit['z'], data_fit['r'], sigma_r, Sigma_0=10*np.eye(2))
posterior = tfd.MultivariateNormalFullCovariance(loc=posterior_params[0], covariance_matrix=posterior_params[1])

# generate samples from gamma posterior
samples = posterior.sample(10)

# generate N dream data sets
dreams = generate_dream_data_set(posterior, T=5, N=10,z_prior_type='informative')

#append episodic memories to dreams

dreams_plus_EM = []
for dream in dreams:
    dream_plus_EM = {'z':np.concatenate([dream['z'], data_EM['z']]), 'r':np.concatenate([dream['r'], data_EM['r']])}
    dream_plus_EM = dreams_plus_EM.append(dream_plus_EM)
dreams = dreams_plus_EM
    
# mllh on 2D 1 task model
onetask_mllhs_dream = [helper.model_marginal_llh_analytic(dream['z'], dream['r'], sigma_r, Sigma_0=np.eye(2)) for dream in dreams]

# mllh on 2D 2 task model
twotask_mllhs_dream = [model_marginal_llh_analytic_2x2D(dream['z'], dream['r'], sigma_r, Sigma_0_2D = np.array([[1., 0.], [0., 1.]])) for dream in dreams]

# mllhs on ground truth data
onetask_mllh = helper.model_marginal_llh_analytic(data_fit['z'], data_fit['r'], sigma_r, Sigma_0=np.eye(2))
twotask_mllh = model_marginal_llh_analytic_2x2D(data_fit['z'], data_fit['r'], sigma_r, Sigma_0_2D = np.array([[1., 0.], [0., 1.]]))

In [ ]:
plt.figure(figsize=(10,3))
plt.subplots_adjust(wspace=0.2, hspace=0.4)

plt.subplot(1,2,1)
plot_data_xy_labels(true_data, xylabels)
plt.title("true data")

plt.subplot(1,2,2)
samples = posterior.sample(300)
sns.kdeplot(x=samples[:, 0], y=samples[:, 1])
plt.scatter(x=samples[:, 0], y=samples[:, 1])
plt.title("gamma posterior")


In [ ]:
plt.figure(figsize=(10,10))
plt.subplots_adjust(wspace=0.4, hspace=0.4)

plt.subplot(3,1,1)
plt.hist(onetask_mllhs_dream,100)
plt.axvline(onetask_mllh, color='gray', linestyle='dashed', linewidth=1)
plt.axvline(np.mean(onetask_mllhs_dream), color='lightblue', linestyle='dashed', linewidth=1)
plt.xlabel("mllh value, true (black dashed), avg (blue dashed)")
plt.ylabel("occurences")

plt.hist(twotask_mllhs_dream,100)
plt.axvline(twotask_mllh, color='k', linestyle='dashed', linewidth=1)
plt.axvline(np.mean(twotask_mllhs_dream), color='darkblue', linestyle='dashed', linewidth=1)
plt.title("both tasks, 2 task (dark), 1 task (light)")

plt.subplot(3,2,3)
plt.hist(onetask_mllhs_dream,100)
plt.axvline(onetask_mllh, color='k', linestyle='dashed', linewidth=1)
plt.axvline(np.mean(onetask_mllhs_dream), color='b', linestyle='dashed', linewidth=1)
plt.xlabel("mllh value, true (black dashed), avg (blue dashed)")
plt.ylabel("occurences")
plt.title("1 x 2D task")

plt.subplot(3,2,4)
plt.hist(twotask_mllhs_dream,100)
plt.axvline(twotask_mllh, color='k', linestyle='dashed', linewidth=1)
plt.axvline(np.mean(twotask_mllhs_dream), color='b', linestyle='dashed', linewidth=1)
plt.title("2 x 2D task")

plt.subplot(3,3,7)
plt.plot(["1x2D","2x2D"],[onetask_mllh, twotask_mllh])
plt.plot(["1x2D","2x2D"],[np.mean(onetask_mllhs_dream), np.mean(twotask_mllhs_dream)])
plt.legend(["true","avg dream"])

plt.subplot(3,3,8)
plt.bar(["1x2D","2x2D"],[onetask_mllh, twotask_mllh])
plt.title("mllh on true data")

plt.subplot(3,3,9)
plt.bar(["1x2D","2x2D"],[np.mean(onetask_mllhs_dream), np.mean(twotask_mllhs_dream)])
plt.title("avg mllh on dream data")


In [ ]:
#num_subplots = len(dreams)
num_subplots = 5
plt.figure(figsize=(15,10))
for i in range(num_subplots):
    plt.subplot(num_subplots, num_subplots,i+1)
    helper.plot_data(dreams[i], labels=False)

In [ ]:
#num_subplots = len(dreams)
num_subplots = 5
plt.figure(figsize=(15,10))
for i in range(num_subplots):
    plt.subplot(num_subplots, num_subplots,i+1)
    plt.scatter(dreams[i]['z'].T[0],dreams[i]['z'].T[1])

In [ ]:
points = data_fit
plt.scatter(points['z'].T[0],points['z'].T[1])

In [ ]:
points = data_x
plt.scatter(points['z'].T[0],points['z'].T[1])
plt.xlim([-1.5,1.5])
plt.ylim([-1.5,1.5])

points = data_EM
plt.scatter(points['z'].T[0],points['z'].T[1])

points = dreams[0]
plt.scatter(points['z'].T[0],points['z'].T[1],alpha = 0.5)

plt.legend(["task 1", "EM contents", "dreamed"])